In [2]:
!pip install -r requirements.txt

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 358kB 75.0MB/s eta 0:00:01
     |████████████████████████████████| 421.8MB 8.1MB/s eta 0:00:0101s eta 0:00:155.6MB 29.4MB/s eta 0:00:14.2MB 29.4MB/s eta 0:00:14141414| 42.1MB 29.4MB/s eta 0:00:13                         | 46.6MB 29.4MB/s eta 0:00:13                         | 51.0MB 29.4MB/s eta 0:00:13.8MB 29.4MB/s eta 0:00:13                     | 61.3MB 29.4MB/s eta 0:00:13               | 66.3MB 29.4MB/s eta 0:00:13��█▍                          | 70.9MB 102.2MB/s eta 0:00:04MB/s eta 0:00:04  | 78.7MB 102.2MB/s eta 0:00:04██▍                         | 84.2MB 102.2MB/s eta 0:00:04        | 89.2MB 102.2MB/s eta 0:00:04��█                         | 93.5MB 102.2MB/s eta 0:00:04MB 102.2MB/s eta 0:00:04                | 102.7MB 102.2MB/s eta 0:00:04                | 106.5MB 102.2MB/s eta 0:00:04040403               | 132.5MB 102.2MB/s eta 0:00:03��██▍                     | 136.8

     |████████████████████████████████| 1.3MB 51.7MB/s eta 0:00:01��████▋                     | 419kB 51.7MB/s eta 0:00:01
     |████████████████████████████████| 450kB 43.5MB/s eta 0:00:01/s eta 0:00:01
     |████████████████████████████████| 3.8MB 53.4MB/s eta 0:00:01s eta 0:00:01
     |████████████████████████████████| 81kB 77.6MB/s eta 0:00:01
     |████████████████████████████████| 133kB 82.0MB/s eta 0:00:01
     |████████████████████████████████| 153kB 67.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 76.8MB/s eta 0:00:01
ERROR: modelarts 1.1.3 has requirement Flask==1.0.2, but you'll have flask 0.12.2 which is incompatible.
ERROR: modelarts 1.1.3 has requirement Flask-Cors==3.0.4, but you'll have flask-cors 3.0.3 which is incompatible.
ERROR: modelarts 1.1.3 has requirement psutil==5.4.6, but you'll have psutil 5.4.3 which is incompatible.
ERROR: modelarts 1.1.3 has requirement urllib3==1.21.1, but you'll have urllib3 1.22 which is incompatible.
ERROR: tensorflow

      Successfully uninstalled requests-2.18.4
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
  Found existing installation: imageio 2.2.0
ERROR: Cannot uninstall 'imageio'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [ ]:
!pip install --upgrade pip 

In [ ]:
!pip uninstall -y PyWavelets
!pip install PyWavelets

In [ ]:
!pip uninstall -y Pillow
!pip install Pillow

In [ ]:
import os
import time
import random
import numpy as np
import scipy, multiprocessing
import tensorflow as tf
import tensorlayer as tl
from model import get_G, get_D
from config import config
from PIL import Image

import math
from random import randrange

import pandas as pd

import matplotlib.pyplot as plt

from skimage import img_as_float
from skimage.measure import compare_ssim as ssim, compare_psnr as psnr

import json

In [ ]:
def evaluate(checkpoint_dir, model, valid_lr_img, valid_hr_img, image_name, G = None, save_dir = "validation-samples"):

    os.makedirs(save_dir, exist_ok=True)
    
    valid_lr_img = (valid_lr_img / 127.5) - 1  # rescale to ［－1, 1]

    if not G:
        G = get_G([1, None, None, 3])
        G.load_weights(os.path.join(checkpoint_dir, model))
        G.eval()

    valid_lr_img = np.asarray(valid_lr_img, dtype=np.float32)
    valid_lr_img = valid_lr_img[np.newaxis,:,:,:]
    size = [valid_lr_img.shape[1], valid_lr_img.shape[2]]

    out = G(valid_lr_img).numpy()
    
    model_num = model.replace(".h5","").split("-")[1]

    print("LR size: %s /  generated HR size: %s" % (size, out.shape))  # LR size: (339, 510, 3) /  gen HR size: (1, 1356, 2040, 3)
    
    if not os.path.isfile('sr-' + model_num + "-" + image_name):
        tl.vis.save_image(out[0], os.path.join(save_dir, 'sr-' + model_num + "-" + image_name))

        out_bicu = scipy.misc.imresize(valid_lr_img[0], [size[0] * 4, size[1] * 4], interp='bicubic', mode=None)
        tl.vis.save_image(out_bicu, os.path.join(save_dir, 'bic-' + model_num + "-" + image_name))

    sr_smaller = tf.image.resize(out[0], size=size)
    hr_smaller = tf.image.resize(valid_hr_img, size=size)

    validate = {
        "sr" : out[0],
        "sr_resized" : sr_smaller.numpy(),
        
        "lr" : valid_lr_img[0],
        "bic" : out_bicu,
        
        "hr" : valid_hr_img, 
        "hr_resized" : hr_smaller.numpy(),
    }
    
    data = {
        "G" : G,
        
        "model" : model,

        "psnr_lr" : psnr( validate.get("lr"),  validate.get("sr_resized")),
        "ssim_lr" : ssim(validate.get("lr"),  validate.get("sr_resized"), multichannel=True),

        "psnr_hr_4" : psnr( validate.get("hr_resized"),  validate.get("sr_resized"), data_range = 255),
        "ssim_hr_4" : ssim(validate.get("hr_resized"),  validate.get("sr_resized"), multichannel=True),
        
        "psnr_hr" : psnr( validate.get("hr"),  validate.get("sr")),
        "ssim_hr" : ssim(validate.get("hr"),  validate.get("sr"), multichannel=True),

        "psnr_bic_hr" : psnr( validate.get("hr"),  validate.get("bic")),
        "ssim_bic_hr" : ssim( validate.get("hr"),  validate.get("bic"), multichannel=True),
    }
    return data
       

In [ ]:
def evaluate_downsample(checkpoint_dir, model, valid_hr_img, image_name, G = None, save_dir = "validation-ds-samples"):

    os.makedirs(save_dir, exist_ok=True)
    
    size = [int(valid_hr_img.shape[0]/4), int(valid_hr_img.shape[1]/4)]
    
    hr_smaller = tf.image.resize(valid_hr_img, size=size)
    
    valid_lr_img = (hr_smaller / 127.5) - 1  # rescale to ［－1, 1]

    if not G:
        G = get_G([1, None, None, 3])
        G.load_weights(os.path.join(checkpoint_dir, model))
        G.eval()

    valid_lr_img = np.asarray(valid_lr_img, dtype=np.float32)
    valid_lr_img = valid_lr_img[np.newaxis,:,:,:]
    

    out = G(valid_lr_img).numpy()
    
    model_num = model.replace(".h5","").split("-")[1]

    print("LR size: %s /  generated HR size: %s" % (size, out.shape))  # LR size: (339, 510, 3) /  gen HR size: (1, 1356, 2040, 3)
    
    if not os.path.isfile('sr-' + model_num + "-" + image_name):
        tl.vis.save_image(out[0], os.path.join(save_dir, 'sr-' + model_num + "-" + image_name))

        out_bicu = scipy.misc.imresize(valid_lr_img[0], [size[0] * 4, size[1] * 4], interp='bicubic', mode=None)
        tl.vis.save_image(out_bicu, os.path.join(save_dir, 'bic-' + model_num + "-" + image_name))

    sr_smaller = tf.image.resize(out[0], size=size)
    

    validate = {
        "sr" : out[0],
        "sr_resized" : sr_smaller.numpy(),
        
        "lr" : valid_lr_img[0],
        "bic" : out_bicu,
        
        "hr" : valid_hr_img, 
        "hr_resized" : hr_smaller.numpy(),
    }
    
    data = {
        "G" : G,
        
        "model" : model,

        "psnr_lr" : psnr( validate.get("lr"),  validate.get("sr_resized")),
        "ssim_lr" : ssim(validate.get("lr"),  validate.get("sr_resized"), multichannel=True),

        "psnr_hr_4" : psnr( validate.get("hr_resized"),  validate.get("sr_resized"), data_range = 255),
        "ssim_hr_4" : ssim(validate.get("hr_resized"),  validate.get("sr_resized"), multichannel=True),
        
        "psnr_hr" : psnr( validate.get("hr"),  validate.get("sr")),
        "ssim_hr" : ssim(validate.get("hr"),  validate.get("sr"), multichannel=True),

        "psnr_bic_hr" : psnr( validate.get("hr"),  validate.get("bic")),
        "ssim_bic_hr" : ssim( validate.get("hr"),  validate.get("bic"), multichannel=True),
    }
    return data
       

In [ ]:
###====================== PRE-LOAD DATA ===========================###
valid_hr_img_list = sorted(tl.files.load_file_list(path=config.VALID.hr_img_path, regx='.*.png', printable=False))
valid_lr_img_list = sorted(tl.files.load_file_list(path=config.VALID.lr_img_path, regx='.*.png', printable=False))

valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, path=config.VALID.lr_img_path, n_threads=32)

valid_hr_imgs = tl.vis.read_images(valid_hr_img_list, path=config.VALID.hr_img_path, n_threads=32)

In [ ]:
def createPyPlot(validate_data, resized = True):


    label = 'SSIM: {:.2f}, sk_psnr:{:.2f} PSNR: {:.2f}'

    if resized: # show the images at size == the size of the input LR image
        fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(17, 12),
                             sharex=True, sharey=True)
        ax = axes.ravel()
        
        ax[0].imshow(validate_data.get("images").get("lr"))
        ax[0].set_xlabel(label.format(1.00, 100.0, 100.0))
        ax[0].set_title('valid LR image')
        
        ax[1].imshow(validate_data.get("images").get("sr_resized"))
        ax[1].set_xlabel(label.format(validate_data.get("ssim_lr"), validate_data.get("psnr_lr"), validate_data.get("PSNR_lr")))
        ax[1].set_title('generated image resized *-4 vs LR image')
        
        ax[2].imshow(validate_data.get("images").get("hr_resized"))
        ax[2].set_xlabel(label.format(1.00, 100.0, 100.0))
        ax[2].set_title('valid HR resized *-4')      
        
        ax[3].imshow(validate_data.get("images").get("sr_resized"))
        ax[3].set_xlabel(label.format(validate_data.get("ssim_hr_4"), validate_data.get("psnr_hr_4"), validate_data.get("PSNR_hr_4")))
        ax[3].set_title('generated image resized *-4 vs HR resized')
        
    else: 
        
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(17, 12),
                             sharex=True, sharey=True)
        ax = axes.ravel()
    
        ax[0].imshow(validate_data.get("images").get("hr"))
        ax[0].set_xlabel(label.format(1.00, 100.0, 100.0))
        ax[0].set_title('valid HR image')

        ax[1].imshow(validate_data.get("images").get("bic"))
        ax[1].set_xlabel(label.format(validate_data.get("ssim_bic_hr"), validate_data.get("psnr_bic_hr"), validate_data.get("PSNR_bic_hr")))
        ax[1].set_title('bicubic interpolation *4 vs HR')

        ax[2].imshow(validate_data.get("images").get("sr"))
        ax[2].set_xlabel(label.format(validate_data.get("ssim_hr"), validate_data.get("psnr_hr"), validate_data.get("PSNR_bic_hr")))
        ax[2].set_title('generated image vs HR')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def compare_models_names(a):
    return int(a.replace(".h5","").split("-")[1])

In [ ]:
def rand_three(l):
    return [i for i in set((randrange(l), randrange(l), randrange(l), randrange(l), randrange(l)))][:3]

rand_three(100)

In [ ]:
models = sorted(tl.files.load_file_list(path="checkpoint", regx='g-[0-9]+\.(h5)', printable=False), key=compare_models_names)
pd.DataFrame(models).head()

In [ ]:
G = None

l = len(valid_hr_img_list)

for image in rand_three(l):
    validate_array = []
    for model in models:
        valid_lr_img = valid_lr_imgs[image]
        valid_hr_img = valid_hr_imgs[image]
        image_name = valid_hr_img_list[image]
        
        ev = evaluate("checkpoint", model, valid_lr_img, valid_hr_img, image_name, G = G)
        
        G = ev.pop("G", G)
        validate_array.append(ev) 
        
    with open("logs/" + image_name + ".json", mode='w', encoding='utf-8') as f:
        json.dump(validate_array, f)
    

In [ ]:
l = len(valid_hr_img_list)

for image in rand_three(l):
    validate_ds_array = []
    for model in models:
        valid_hr_img = valid_hr_imgs[image]
        image_name = valid_hr_img_list[image]
        
        ev = evaluate_downsample("checkpoint", model, valid_hr_img, image_name, G = G)
        
        G = ev.pop("G", G)
        validate_ds_array.append(ev) 
        
    with open("logs/" + image_name + "-ds.json", mode='w', encoding='utf-8') as f:
        json.dump(validate_ds_array, f)
    

In [ ]:
pd.DataFrame(validate_ds_array)